In [1]:
# ! pip install selenium

In [98]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC 
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.action_chains import ActionChains

from time import sleep 
from datetime import datetime, date
from openpyxl import load_workbook # для работы с excel

In [95]:
# Параметры поездки

# Откуда
origin = 'москва'
# Куда
destination = 'махачкала'

# Дата отправления
# Month выбрать из ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
month_departure = "Февраль"
# Day выбрать из ['1', '2', '3', ..., '28', '29', '30', '31']
day_departure = '14'

# Дата возвращения
# Month выбрать из ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
month_return = "Февраль"
# Day выбрать из ['1', '2', '3', ..., '28', '29', '30', '31']
day_return = '16'

# Введите желаемое время прилета в город командировки формате 'hh:00'
arrival_start_time = '08:00'
arrival_end_time = '11:00'
# Введите желаемое время прилета обратно в Москву из города командировки формате 'hh:00'
returning_start_time = '19:00'
returning_end_time = '00:00'

In [99]:
# Save our trip data to Excel report file
wb = load_workbook('C:\Evhen\BSNS\ViK_Medical\Отчеты.xlsx')
ws = wb['Sheet1']

city_col = 3
date_col = 5
rows = [x for x in range (2, 150)]

# Формируем дату для отчета
def get_month(month):
    m = str(months.index(month)+1)
    if len(m)<2:
        m = '0' + m
    return m

months = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
year = str(date.today().year)
dt = day_return + '.' + get_month(month_return) + '.' + year

for row in rows:
    # fill first empty row
    if (ws.cell(row=row, column=city_col).value==None) and (ws.cell(row=row, column=date_col).value==None):
        ws.cell(row=row, column=city_col).value = destination.capitalize()
        ws.cell(row=row, column=date_col).value = dt
        break

wb.save('C:\Evhen\BSNS\ViK_Medical\Отчеты.xlsx')

In [3]:
# to exclude DeprecationWarning
service = Service('C:/Users/eadub/chromedriver_win32/chromedriver.exe')
# Запускаем Chrome driver
browser = Chrome(service=service)
browser.maximize_window()
browser.implicitly_wait(10)

In [5]:
url = 'https://www.aviasales.ru'
browser.get(url)
sleep(5)

In [7]:
# Снимаем галочку c Ostrovok, чтобы скрипт не открывал лишние окна
hotels_cancel = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".of_input_checkbox__label")))
hotels_cancel.click()

In [8]:
# Заполняем поле Куда
input_destination = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#destination")))
input_destination.click()
input_destination.clear()

input_destination.send_keys(destination)
input_destination.click()
sleep(2)

In [9]:
# Заполняем поле Откуда
# Изначально поле From заполнено текущим местположением, которое Aviasales определяет по ip, поэтому нам нужно 
# сначала очистить это поле. Для этого нужно использовать WebDriverWait с expected_conditions настройкой element_to_be_clickable
# clear the search text box
input_origin = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#origin")))
input_origin.click()
input_origin.clear()
# sleep(2)
input_origin.clear()
# input_origin.click()
# send text to the clear field
# input_origin = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#origin")))
input_origin.send_keys(origin)
input_origin.click()
sleep(2)

In [10]:
# Заполняем дату отправления
input_departure_date = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.trip-duration__field:nth-child(1) > div:nth-child(1)")))
# Клик по полю Дата отправления
input_departure_date.click()
sleep(2)

# Клик по Месяцу, чтобы выбрать месяц отправления
month_selection_dep = Select(browser.find_element(By.CSS_SELECTOR, "div.calendar__month:nth-child(1) > div:nth-child(1) > div:nth-child(1) > select:nth-child(2)"))
sleep(2)
month_selection_dep.select_by_visible_text(month_departure)

# Выбор дня отправления
calendar_dep = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.calendar__month:nth-child(1)")))
dates_dep = calendar_dep.find_elements(By.CLASS_NAME, "calendar-day__date")

# Немного прокрутим страницу вниз, чтобы был виден всеь календарь целиком
browser.execute_script("window.scrollTo(0, 200)")

dep_day = dates_dep[int(day_departure)-1]
dep_day.click()

In [11]:
# Заполняем дату возвращения                                                                 
input_return_date = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.--return:nth-child(1)")))
# Клик по полю Дата отправления
input_return_date.click()
sleep(2)
                                                                    
# Клик по Месяцу, чтобы выбрать месяц возвращения                   
month_selection_ret = Select(browser.find_element(By.CSS_SELECTOR, "div.calendar__month:nth-child(2) > div:nth-child(1) > div:nth-child(1) > select:nth-child(2)"))
month_selection_ret.select_by_visible_text(month_return)
sleep(2)

# Выбор дня возвращения                                                              
calendar_ret = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.calendar__month:nth-child(2)")))
dates_ret = calendar_ret.find_elements(By.CLASS_NAME, "calendar-day__date")

# Немного прокрутим страницу вниз, чтобы был виден всеь календарь целиком
# browser.execute_script("window.scrollTo(0, 200)")

ret_day = dates_ret[int(day_return)-1]
ret_day.click()

In [12]:
# Опция
# Выбираем количество пассажиров и класс
# Клик по форме для выбора пассажиров и класса 
input_passengers_class = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".additional-fields")))
input_passengers_class.click()

In [13]:
# Опция
# Выбираем количество пассажиров и класс

# All our workers are adults, so we only need field for adults amount
num_passengers = browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text

# Сколько пассажиров полетит 
# (В нашей компании всего 3 человека регулярно совершают командировки. Чаще всего летит кто-то один)
passengers = 1

while int(num_passengers) < passengers:
    num_passengers = int(browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text)
    plus_button = browser.find_element(By.CSS_SELECTOR, "div.additional-fields__passenger-row:nth-child(1) > div:nth-child(2) > div:nth-child(3) > a:nth-child(1) > svg:nth-child(1) > path:nth-child(1)")
    plus_button.click()
    num_passengers = browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text
    # 9 - is maximum amount of passengers for one search
    if int(num_passengers) == 9:
        num_passengers = int(browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text)
        break
        
while int(num_passengers) > passengers:
    num_passengers = int(browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text)
    minus_button = browser.find_element(By.CSS_SELECTOR, "div.additional-fields__passenger-row:nth-child(1) > div:nth-child(2) > div:nth-child(1) > a:nth-child(1)")
    minus_button.click()
    num_passengers = browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text
    if int(num_passengers) == 1:
        num_passengers = int(browser.find_element(By.CLASS_NAME, "additional-fields__passenger-value").text)
        break
        
# Выберите класс (cl) из ['Economy', 'Business']
cl = 'Economy'

class_selection = browser.find_elements(By.CLASS_NAME, "custom-radio__caption")
class_button = browser.find_elements(By.CLASS_NAME, "custom-radio__element")

if cl == class_selection[0].text:
    class_button[0].click()
elif cl == class_selection[1].text:
    class_button[1].click()

In [14]:
# Start searching
search_button = browser.find_element(By.CSS_SELECTOR, ".button_70e8ad4")
search_button.click()
sleep(10)

In [15]:
# Working with tickets parameters

In [16]:
# # Working with tickets parameters

# # Choosing currency (we need RUB)
# currency = 'RUB'
# currency_button = browser.find_element(By.CSS_SELECTOR, ".q8JaLokdctjkb1_n4aw0 > svg:nth-child(1)")
# currency_button.click()

# currency_selection = browser.find_element(By.CSS_SELECTOR, "li.xHo38THtncBgNdQpw5i3:nth-child(14) > p:nth-child(1)")
# # Moving down to needed value on drop-down list
# actions = ActionChains(browser)
# actions.move_to_element(currency_selection).perform()
# currency_selection = wait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "li.xHo38THtncBgNdQpw5i3:nth-child(14) > p:nth-child(1)")))
# currency_selection.click()
# currency_selection_apply = wait(browser, 5).until(EC.visibility_of_element_located((By.CSS_SELECTOR, "button.button_278c41a:nth-child(2)")))
# currency_selection_apply.click()

In [17]:
def check_flights():
    try:
        no_flights = browser.find_element(By.CSS_SELECTOR, ".error-informer__container > h2:nth-child(1)")    
        if no_flights:
            print("Билеты не найдены")
            print("Попробуйте поменять даты")
    except Exception as e:
        pass

In [18]:
check_flights()

In [19]:
# Choosing only direct flights
direct_flight_option = browser.find_element(By.CSS_SELECTOR, 'div.filter-group:nth-child(1) > div:nth-child(2) > div:nth-child(1) > label:nth-child(1) > span:nth-child(2)')
direct_flight_option.click()
# Ставим галочку Без ночных пересадок
no_night_transfers = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.filter-group:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(3) > label:nth-child(1) > span:nth-child(2)")))
no_night_transfers.click()

In [20]:
arrival_start_time = datetime.strptime(arrival_start_time, '%H:%M').time()
arrival_end_time = datetime.strptime(arrival_end_time, '%H:%M').time()

returning_start_time = datetime.strptime(returning_start_time, '%H:%M').time()
returning_end_time = datetime.strptime(returning_end_time, '%H:%M').time()

In [21]:
# Adjusting arrival time with sccroller
# Scrolling down a little bit
browser.execute_script("window.scrollTo(0, 200)")

departure_time_menu = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(4) > div:nth-child(1) > div:nth-child(1)")
departure_time_menu.click()
sleep(2)

In [22]:
# Setting arrival time functions
def set_start_time_arrvl(goal_time, browser_time):
    if goal_time.hour != browser_time.hour:
        # Moving left scroller
        step = 10 * goal_time.hour
        ActionChains(browser).drag_and_drop_by_offset(left_scroller, step, 0).perform()
        
def set_end_time_arrvl(goal_time, browser_time):
    if goal_time.hour != browser_time.hour:
        # Moving right scroller
        step = 10 * goal_time.hour
        ActionChains(browser).drag_and_drop_by_offset(right_scroller, -245+step, 0).perform()

In [23]:
# Adjusting arrival time with sccroller

# Min time
start_time = datetime.strptime(browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text[0:5], 
                               '%H:%M').time()
# Initialize left scroller
left_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(4)")
# Setting arrival start time
set_start_time_arrvl(arrival_start_time, start_time)

# Max time
end_time = datetime.strptime(browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text[-5:], 
                               '%H:%M').time()
# Initialize right scroller
right_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(5)")
# Setting arrival end time
set_end_time_arrvl(arrival_end_time, end_time)

sleep(2)

In [25]:
# Adjusting return time with sccroller
# Scrolling down a little bit
browser.execute_script("window.scrollTo(0, 400)")
returning_time_menu = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(1) > div:nth-child(1)")
returning_time_menu.click()
# Scrolling down a little bit
browser.execute_script("window.scrollTo(0, 600)")

In [26]:
# Setting returning time functions
def set_start_time_ret(goal_time, browser_time):
    if goal_time.hour != browser_time.hour:
        # Moving left scroller
        step = 10 * goal_time.hour
        ActionChains(browser).drag_and_drop_by_offset(left_scroller, step+5, 0).perform()
        
def set_end_time_ret(goal_time, browser_time):
    if goal_time.hour != browser_time.hour:
        # Moving right scroller
        step = 10 * goal_time.hour
        ActionChains(browser).drag_and_drop_by_offset(right_scroller, -240+step, 0).perform()

In [27]:
# Adjusting return time with sccroller
# Min time                                                            
start_time = datetime.strptime(browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text[0:5], 
                               '%H:%M').time()
# Initialize left scroller 
left_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(4)")
# Setting returning start time
set_start_time_ret(returning_start_time, start_time)

# Max time
end_time = datetime.strptime(browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text[-5:], 
                               '%H:%M').time()
# Initialize right scroller
right_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(5)")
# Setting returning end time
set_end_time_ret(returning_end_time, end_time)

sleep(5)

In [28]:
# while returning_start_time != start_time:
#     # Inintialize left scroller 
#     left_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(4)")
#     # Moving left scroller by one hour
#     ActionChains(browser).drag_and_drop_by_offset(left_scroller, 10, 0).perform()
#     start_time = datetime.strptime(browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text[0:5], 
#                                '%H:%M').time()
    
# while returning_end_time != end_time:
#     # Inintialize right scroller 
#     right_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(5)")
#     # Moving right scroller by one hour
#     ActionChains(browser).drag_and_drop_by_offset(right_scroller, -10, 0).perform()
#     end_time = datetime.strptime(browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(5) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(2) > div:nth-child(1)").text[-5:], 
#                                '%H:%M').time()

In [29]:
# def move_arrival_time_scroller_right():
#     browser.execute_script("window.scrollTo(0, 200)")
#     right_scroller = browser.find_element(By.CSS_SELECTOR, "div.filter-group:nth-child(4) > div:nth-child(2) > div:nth-child(2) > div:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(5)")
#     ActionChains(browser).drag_and_drop_by_offset(right_scroller, 10, 0).perform()

In [30]:
def check_filters():
    rigid_filters = browser.find_element(By.CSS_SELECTOR, ".title-2-bold_03dd9f9")
    if rigid_filters:
        # Choosing only direct flights - Убираем галочку
        direct_flight_option = browser.find_element(By.CSS_SELECTOR, 'div.filter-group:nth-child(1) > div:nth-child(2) > div:nth-child(1) > label:nth-child(1) > span:nth-child(2)')
        direct_flight_option.click()
        sleep(2)
        # Убираем галочку Без ночных пересадок
        no_night_transfers = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.filter-group:nth-child(3) > div:nth-child(2) > div:nth-child(1) > div:nth-child(3) > label:nth-child(1) > span:nth-child(2)")))
        no_night_transfers.click()
        sleep(2)
    if rigid_filters:
        loosen_filter_button = browser.find_element(By.CSS_SELECTOR, "button.button_70e8ad4:nth-child(4)")
        loosen_filter_button.click()
        sleep(2)

In [34]:
# Choosing best flight
def choose_flight():
    browser.execute_script("window.scrollTo(0, 100)")
    try:
        best_flight = browser.find_element(By.CSS_SELECTOR, "div.product-list__item:nth-child(2) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(1) > div:nth-child(3) > div:nth-child(1) > div:nth-child(1) > button:nth-child(1)")
        best_flight.click()
        sleep(3)
        
        try:
            best_ticket = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "a.orange_70e8ad4")))
            best_ticket.click()
        except: # если всего один продавец, кнопка перехода к покупке другая
            best_ticket = wait(browser, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "div.buy-button:nth-child(4) > div:nth-child(1) > a:nth-child(1)")))    
            best_ticket.click()
    
    except Exception as e:
        # Если подходящего билета нет, ослабляем фильтры
        check_filters()
        sleep(3)
        choose_flight()

In [35]:
choose_flight()